In [ ]:
# ------------------------------------------------ #
#                  SETUP PATHS                     #
# ------------------------------------------------ #
import os
import sys

# Set project root and scripts directory
project_root = os.path.dirname(os.getcwd())
scripts_dir = os.path.join(project_root, 'Scripts')

# Ensure the Scripts directory is in the Python path
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)


In [2]:
# ------------------------------------------------ #
#                  IMPORT FUNCTIONS                #
# ------------------------------------------------ #
import os
import sys
import datetime

# Get the project root directory (going up one level from Notebooks)
project_root = os.path.dirname(os.getcwd())
# Add the Scripts directory to Python path so we can import modules from individual script files
scripts_dir = os.path.join(project_root, 'Scripts')
if scripts_dir not in sys.path:
    sys.path.append(scripts_dir)

# Add functions here from Scripts folder
from download_ao_indices import ReturnAOFetch
from enso_indices import ReturnENSOFetch
from breakup_data import CalculateBreakupData
#from breakup_data import CalculateEstimatedBreakup
from bounding_box import CalculateBoundingBox
from raster_temp import CalculateMeanTemperature
from correlation_raster import CompareIndexAndTemperature

# Import correlation functions
from correlationfunctions import (
    calculate_ao_correlation,
    calculate_enso_correlation,
    save_results_to_csv,
    plot_results
)


In [3]:
# ------------------------------------------------ #
#                CLASS DEFINITION                  #
# ------------------------------------------------ #

class StudySite:
    def __init__(self, name):
        self.name = name
        self.breakup_anomaly_data = None  # Dictionary of integer years (e.g. 2000) with following data subkeys: zscore_index, anomaly_days, breakup_date
        self.estimated_breakup_date = None  # The MEAN breakup date for the study site across all years, in datetime format.
        self.enso_indices = None  # Dictionary of integer years (e.g. 2000) with enso_value for each of 12 months before breakup
        self.ao_indices = None  # Dictionary of integer years (e.g. 2000) with ao_value for each of 12 months before breakup
        self.bounding_box = None  # Dictionary of max_lat:value, min_lat:value, max_lon:value, min_lon:value
        self.mean_30d_raster_temps = None  # Dictionary of integer years (e.g. 2000) with mean temperatures for the 30 days prior to mean breakup date
        self.enso_correlations = None  # DataFrame of ENSO correlations across all years
        self.ao_correlations = None  # DataFrame of AO correlations across all years

    def calculate_breakup_data(self, ice_data_filepath, years_of_data):
        """Load ice breakup data for this site from the provided icedata spreadsheet, filtering by years_of_data and the site name."""
        self.breakup_anomaly_data = CalculateBreakupData(years_of_data, ice_data_filepath, self.name)
        self.estimated_breakup_date = CalculateEstimatedBreakup(self.breakup_anomaly_data)

    def fetch_enso(self, enso_data_filepath):
        """Pull in ENSO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup date."""
        self.enso_indices = ReturnENSOFetch(enso_data_filepath, self.estimated_breakup_date)

    def fetch_ao(self, ao_data_filepath):
        """Pull in AO data as dictionary of integer years (e.g. 2000) on a MONTHLY scale for 12 months prior to estimated breakup date."""
        self.ao_indices = ReturnAOFetch(ao_data_filepath, self.estimated_breakup_date)

    def calculate_bounding_box(self, ice_data_filepath):
        """Calculate 100km bounding box based on study site coordinates."""
        self.bounding_box = CalculateBoundingBox(ice_data_filepath, self.name)

    def calculate_mean_raster_temps(self):
        """Calculate mean raster temperatures for the 30 days prior to the mean breakup date."""
        self.mean_30d_raster_temps = CalculateMeanTemperature(self.bounding_box, self.estimated_breakup_date)

    def calculate_ao_correlation(self, ice_data_filepath, ao_data_filepath, project_root, lag=1):
        """Calculate AO correlations for this study site."""
        self.ao_correlations = calculate_ao_correlation(
            site_name=self.name,
            ice_data_filepath=ice_data_filepath,
            ao_data_filepath=ao_data_filepath,
            project_root=project_root,
            lag=lag
        )

    def calculate_enso_correlation(self, ice_data_filepath, enso_data_filepath, project_root, lag=1):
        """Calculate ENSO correlations for this study site."""
        self.enso_correlations = calculate_enso_correlation(
            site_name=self.name,
            ice_data_filepath=ice_data_filepath,
            enso_data_filepath=enso_data_filepath,
            project_root=project_root,
            lag=lag
        )


In [4]:
# ------------------------------------------------ #
#                  PARAMETERS                      #
# ------------------------------------------------ #

# File paths
enso_data_filepath = os.path.join(project_root, "Data", "ENSO_index.rtf")
ao_data_filepath = os.path.join(project_root, "Data", "monthly.ao.index.txt")
ice_data_filepath = os.path.join(project_root, "Data", "ice_data.csv")

# Study site names
study_site_names = ["Stebbins"]  # Replace with actual site names

# Years of data to process
years_of_data = list(range(2000, 2023))

# Results directory
results_dir = os.path.join(project_root, "Results")
os.makedirs(results_dir, exist_ok=True)


In [8]:
# ------------------------------------------------ #
#                  MAIN CODE                       #
# ------------------------------------------------ #

# Create list of StudySite objects
sites = [StudySite(name) for name in study_site_names]

# Process data for each site
for site in sites:
    print(f"Processing site: {site.name}")
    
    # Calculate bounding box for the site
    site.calculate_bounding_box(ice_data_filepath)
    
    # Fetch ENSO and AO indices
    site.fetch_enso(enso_data_filepath)
    site.fetch_ao(ao_data_filepath)
    
    # Calculate mean raster temperatures
    site.calculate_mean_raster_temps()
    
    # Perform AO and ENSO correlation calculations
    site.calculate_ao_correlation(
        ice_data_filepath=ice_data_filepath,
        ao_data_filepath=ao_data_filepath,
        project_root=project_root,
        lag=1
    )
    site.calculate_enso_correlation(
        ice_data_filepath=ice_data_filepath,
        enso_data_filepath=enso_data_filepath,
        project_root=project_root,
        lag=1
    )
    print(f"Finished processing site: {site.name}")


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
# ------------------------------------------------ #
#       CORRELATION AND PLOTTING (Optional)        #
# ------------------------------------------------ #

# Save and plot AO and ENSO correlations for each site
for site in sites:
    if site.ao_correlations is not None:
        # Save AO correlations to CSV
        ao_csv_path = os.path.join(results_dir, f"ao_correlation_{site.name}.csv")
        save_results_to_csv(site.ao_correlations, ao_csv_path)
        
        # Plot AO correlations
        ao_plot_dir = os.path.join(results_dir, "AO_Plots")
        plot_results(site.ao_correlations, "AO", ao_plot_dir)
    
    if site.enso_correlations is not None:
        # Save ENSO correlations to CSV
        enso_csv_path = os.path.join(results_dir, f"enso_correlation_{site.name}.csv")
        save_results_to_csv(site.enso_correlations, enso_csv_path)
        
        # Plot ENSO correlations
        enso_plot_dir = os.path.join(results_dir, "ENSO_Plots")
        plot_results(site.enso_correlations, "ENSO", enso_plot_dir)
